# Examples

In [1]:
from xardano import Wallet, XardanoLedger, PlainUtxo, CNA, ONE_XADA, ONE_ZIGHT, debug, ensure, AbstractContract

not yet thawed
contract not found b'\x8e\xf4^\xc5\xc89\xc5u\xe4xB\xe2\x05\x1aG\xa7K\tg=\xd9\x10\xaa\xdb\xedD\x95U\xa8\xf9J\xfd'
allocation already claimed


## Simple Transfers

In [ ]:
ledger = XardanoLedger()
alice = Wallet(ledger)
bob = Wallet(ledger)

In [2]:
ledger.add_utxo(PlainUtxo(alice.pk, {CNA.XLOV : 10000 * ONE_XADA}))
alice.balance()

alice.send({bob.pk : {CNA.XLOV : 10 * ONE_XADA}})
bob.send({alice.pk : {CNA.XLOV : .1 * ONE_XADA}})

alice.balance(), bob.balance()

KeyError: b'\xc5\xaa\x18\xbeme\x17\x10\x9ajm\xf8+\xbb\x19{\xd0\xe2\xf2Sfj:\xc6<\x87\xf9mh\xb0\xfdK'

## Contracts

### PushPop Example

In [ ]:
class PushPop(AbstractContract):
  def __init__(self) -> None:
    super().__init__()
    self.stack = []
    self.limit = 1
  def push(self, x, tithe: TokenValue):
    debug(f'**** PUSH {x}')
    ensure(len(self.stack) < self.limit, "stack overflow")
    price = 2 * self.limit * ONE_XADA
    #match tithe:
    #  case TokenValue(CNA.XLOV, price):
    #    self.stack.append(x)
    #  case _:
    #    raise InsufficientFunds()
    self.stack.append(x)
    return len(self.stack)
  def pop(self, pk):
    debug(f'**** POP')
    ensure(len(self.stack) > 0, "stack underflow")
    x = self.stack.pop()
    self.ctx.send(pk, {CNA.XLOV : self.limit * ONE_XADA})
    return x

In [ ]:
101000000 / ONE_XADA, alice.balance()[CNA.XLOV] / ONE_XADA

(101.0, 9989.1)

In [ ]:
ledger = XardanoLedger()
alice = Wallet(ledger)
# faucet funding
ledger.add_utxo(PlainUtxo(alice.pk, {CNA.XLOV : 10000 * ONE_XADA}))
# deploy the pp contract
pp = alice.deploy(PushPop(), {CNA.XLOV : MIN_XADA})
ledger.utxos_by_hash[pp]

ContractUtxo(<class '__main__.PushPop'>, {<CNA.XLOV: 0>: 1000000})

In [ ]:
# call the pp contract
alice.call(pp, 'push', 'a', TokenValue(CNA.XLOV, 33 * ONE_XADA))
ledger.utxos_by_hash[pp]

ContractUtxo(<class '__main__.PushPop'>, {<CNA.XLOV: 0>: 34000000})

In [ ]:
try:
  alice.call(pp, 'push', 'a', TokenValue(CNA.XLOV, 2 * ONE_XADA))
  raise Exception('second push should fail')
except ValidationException:
  pass

In [ ]:
alice.call(pp, 'pop', alice.pk)

'a'

In [ ]:
try:
  alice.call(pp, 'pop', alice.pk)
  raise Exception('second push should raise stack underflow')
except ValidationException:
  pass

### Token Allocation Example

In [ ]:
@dataclass
class ToyAlloc:
  dest_addr: str
  num_ztars: int
  thaw_dttm: datetime

class ToyRedemption(AbstractContract):
  def __init__(self, alloc: ToyAlloc) -> None:
    super().__init__()
    self.alloc = alloc
  def release(self):
    debug(f'**** RELEASE {self.alloc.dest_addr}')
    ensure(self.ctx.ddtm >= self.alloc.thaw_dttm, 'not yet thawed')
    toks = {CNA.ZTAR : self.alloc.num_ztars}
    add_tokens(toks, CNA.XLOV, MIN_XADA) # refund
    debug(f'toks to redeemer {toks}')
    self.ctx.send(self.alloc.dest_addr, toks)
    TokenValue(CNA.ZTAR, self.ctx.halt(self.alloc.num_ztars))

class ToyShard(AbstractContract):
  def __init__(self, allocations: dict) -> None:
    super().__init__()
    self.allocations = copy(allocations)
    self.claimed = dict()
  def claim(self, key, claim_deposit):
    debug(f'**** CLAIM {key} allocations {self.allocations} claimed {self.claimed}')
    ensure(key in self.allocations, 'allocation does not exist')
    ensure(key not in self.claimed, 'allocation already claimed')
    debug(f'claim_deposit {claim_deposit}')
    ensure(claim_deposit == xadas(1), 'claim must include deposit for redemption utxo')
    alloc = self.allocations[key]
    debug(f'alloc {alloc}')
    self.claimed[key] = True
    toks = {CNA.ZTAR : alloc.num_ztars}
    add_tokens(toks, CNA.XLOV, MIN_XADA) # include a min_xada for the redemption
    rdm = self.ctx.deploy(ToyRedemption(alloc), toks)
    debug(f'rdm {rdm}')
    return rdm

In [ ]:
ledger = XardanoLedger()
tge = Wallet(ledger)
alice = Wallet(ledger)
bob = Wallet(ledger)
charlie = Wallet(ledger)
# faucet funding
for w in [tge, alice, bob, charlie]:
  ledger.add_utxo(PlainUtxo(w.pk, {CNA.XLOV : 10000 * ONE_XADA}))
# give the TGE some ZTARs
ledger.add_utxo(PlainUtxo(tge.pk, {CNA.ZTAR : 1000000}))
# Invent some allocations
allocs = {
    id(alice)   : ToyAlloc(alice.pk,     10, ledger.time + timedelta(days=10)),
    id(bob)     : ToyAlloc(bob.pk,      100, ledger.time + timedelta(days=11)),
    id(charlie) : ToyAlloc(charlie.pk, 1000, ledger.time + timedelta(days=12)),
}
# deploy the shard contract
min_xadas = 1 # use the following to fund all claims: len(allocs) + 1
shard = tge.deploy(ToyShard(allocs), {CNA.XLOV : min_xadas * MIN_XADA, CNA.ZTAR : 1111})
ledger.utxos_by_hash[shard]

ContractUtxo(<class '__main__.ToyShard'>, {<CNA.XLOV: 0>: 1000000, <CNA.ZTAR: 1>: 1111})

In [ ]:
before = alice.balance()
alice_rdm = alice.call(shard, 'claim', id(alice), xadas(1))
before, alice.balance()

({<CNA.XLOV: 0>: 10000000000}, {<CNA.XLOV: 0>: 9989000000})

Alice cannot release funds from her redemption contract until the thaw period expires.

In [ ]:
try:
  alice.balance(), alice.call(alice_rdm, 'release'), alice.balance()
except Exception as ex:
  print(ex)

not yet thawed


In [ ]:
ledger.sleep(timedelta(days=10))
alice.balance(), alice.call(alice_rdm, 'release'), alice.balance()

({<CNA.XLOV: 0>: 9989000000},
 10,
 {<CNA.XLOV: 0>: 9980000000, <CNA.ZTAR: 1>: 10})

Now the redemption has been released, it no longer exists.

In [ ]:
try:
  alice.balance(), alice.call(alice_rdm, 'release'), alice.balance()
except Exception as ex:
  print(ex)

contract not found b'y-\xb1#Y\x11\x02V{\xa9F\xda[M\xb45\x94\xf4\x93+\xcf\xb5\xde\xc3[\x8c\xbb\xcd\xe3\xd6\xbaL'


And Alice cannot make another claim.

In [ ]:
try:
  alice_rdm = alice.call(shard, 'claim', id(alice), xadas(1))
except Exception as ex:
  print(ex)

allocation already claimed


But the shard is still there, and Bob can still claim.

In [ ]:
before = bob.balance()
bob_rdm = bob.call(shard, 'claim', id(bob), xadas(1))
before, bob.balance()

({<CNA.XLOV: 0>: 10000000000}, {<CNA.XLOV: 0>: 9989000000})

And, as currently defined, there is no access control on the redemption contract.  This is "safe" in the sense that it can deposit tokens only to the predetermined destination address, but may have tax implications for the recipient.

In [ ]:
ledger.sleep(timedelta(days=1))
alice.balance(), bob.balance(), alice.call(bob_rdm, 'release'), alice.balance(), bob.balance()

({<CNA.XLOV: 0>: 9980000000, <CNA.ZTAR: 1>: 10},
 {<CNA.XLOV: 0>: 9989000000},
 100,
 {<CNA.ZTAR: 1>: 10, <CNA.XLOV: 0>: 9970000000},
 {<CNA.XLOV: 0>: 9990000000, <CNA.ZTAR: 1>: 100})